In [2]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [22]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False
    
def hdfc(f):
    try:
        pars = tabula.read_pdf(f,                                            
                               pages='all',
                               silent=True,
                               stream=True,
                               multiple_tables = True,
                               pandas_options={ 'header': None }) 

        df = pd.DataFrame()
        tables = []
        for i,c in enumerate(pars):
            if c.shape[1] == 7:
                tables.append(c)
            elif c.shape[1] == 6:
                c[6]=np.nan; c = c[[0,1,2,3,4,6,5]]; c.columns=range(c.shape[1]);
                tables.append(c)
            elif c.shape[1] == 8:
                c = c[[0,1,3,4,5,6,7]];c.columns = range(c.shape[1]);
                tables.append(c)   
        df = pd.concat(tables, ignore_index=True)

        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
                try:
                    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
                except:
                    print("\nHDFC-Column headers missing"); log.write("\nHDFC-Column headers missing"); pass
                
        try:
            idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('statement summary').any(),axis=1)==True].index][0]    
            df.drop(df.index[idx2:], inplace=True)
        except:pass

        df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
        df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
        df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

        try:
            bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
            df[bal]=df[bal].apply( lambda x: abc(x) )
        except: print("\nBalance columns missing")   

        df['flag']=df.iloc[:,-1].shift(1)
        df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

        try:
            dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
            except:
                try:
                    dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
                except:pass

        try:
            chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
        except:
            try:
                chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
            except:pass

        try:
            narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                    except:
                        try:
                            narr=[c for c in df.columns if "NARRATION" in str(c).upper() ][0]
                        except:pass

        try:
            wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
        except:
            try:
                wdl=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
            except:
                try:
                    wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
                except:pass

        try:
            dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
        except:
            try:
                 dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
            except:pass

        try:
            df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
            df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
            df['Wdl1']=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1
            df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
        except: 
            try:
                Wdl1=[]
                for i, item in enumerate(df[wdl]):
                    try:
                        tmpp=float(str(item).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1
                    except ValueError:
                        df.iloc[i,df.columns.get_loc(wdl)-1]=df.iloc[i,df.columns.get_loc(wdl)]
                        df.iloc[i,df.columns.get_loc(wdl)]=df.iloc[i,df.columns.get_loc(dep)]
                        df.iloc[i,df.columns.get_loc(dep)]=df.iloc[i,df.columns.get_loc(bal)]
                        df.iloc[i,df.columns.get_loc(bal)]=df.iloc[i,df.columns.get_loc(bal)+1]
                    Wdl1.append(tmpp)  
                df[bal]=df[bal].fillna(df.iloc[:,df.columns.get_loc(bal)-1])
                df.drop(df.columns[df.columns.get_loc(bal)+1],axis=1,inplace=True)
                df['Wdl1']=Wdl1   
                df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
            except:pass

        df=df.T.drop_duplicates().T           
        df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
        df['flag']=df.iloc[:,0].astype(str)+df['Wdl1'].astype(str) +df[bal].astype(str)
        df['flag2']=np.arange(len(df))

        df.loc[ df[['flag']].duplicated(keep=False) , 'flag' ] =df['flag'] + df['flag2'].astype(str)  
        df['flag']=df['flag'].apply( lambda row : np.nan if 'nannannan' in row else row ).fillna(method='ffill')        

        df[narr]=df.groupby('flag')[narr].transform( lambda x :' '.join(x))                
        df=df.drop_duplicates(['flag'],keep='first').iloc[0:,:-3].reset_index(drop=True)

        df = df.replace(r'^\s*$', np.nan, regex=True)

        if isnan(df[dep][0]) == False:
            df["Credits"] = np.nan
            df["Credits"][0] = float(str(df[dep][0]).replace(",","").replace("(Cr)","").replace("(Dr)",""))
            df["Debits"] = np.nan
        elif isnan(df[wdl][0]) == False:
            df["Credits"] = np.nan    
            df["Debits"] = np.nan
            df["Debits"][0] = float(str(df[wdl][0]).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1

        for i,j in enumerate(df[bal]):
            if i < len(df[bal])-1 :
                if df[bal][i] < df[bal][i+1]:
                    df["Credits"][i+1] = df[bal][i+1]-df[bal][i]
                elif df[bal][i] > df[bal][i+1]:
                    df["Debits"][i+1] = df[bal][i+1]-df[bal][i]
            else:
                pass

        df = df[[dat,chq,narr,"Debits","Credits",bal]]
        df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]
        df.to_excel("hdfc12.xlsx", index=False)
        return df

    except Exception as e:
        print("Error : ",e)
        pass

f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\HDFC\hdfc\hdfc12.pdf"
try:
    df = hdfc(f) 
    print("Parsed")
except Exception as e:
    print("Error:",e)
    print("Not Parsed")

Parsed


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-